In [48]:
import os
import torch
import logging
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from tqdm.auto import tqdm
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from huggingface_hub import login
from typing import List, Dict, Union, Tuple
from transformers import AutoTokenizer, AutoModel


from beir.retrieval import models
from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

load_dotenv()
login(os.environ["HF_TOKEN"])
print("CUDA_VISIBLE_DEVICES:", os.environ["CUDA_VISIBLE_DEVICES"], "HF_HOME:", os.environ["HF_HOME"])

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout[]

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/mohsenfayyaz/.cache/huggingface/token
Login successful
CUDA_VISIBLE_DEVICES: 4 HF_HOME: /local1/mohsenfayyaz/.hfcache/


In [49]:
class DatasetLoader:
    def __init__(self):
        pass

    def load_dataset(self, dataset_name, use_gold_docs=False) -> dict:
        """
        write docstirng here

        Args:
            dataset_name (str): name of the dataset [re-docred, nq, ...]
            use_gold_docs (bool, optional): To use only gold docs as corpus or not. Defaults to False.
        
        Returns:
            dict: {
                queries: {'test0': 'what is non controlling interest on balance sheet', ...}
                qrels: {'test0': {'doc0': 1, 'doc1': 1}, ...)
                corpus: {'doc0': {'text': "In accou...", 'title': 'Minority interest'}, ...})
            }
        """
        logging.info(f"Loading dataset: {dataset_name}")
        if dataset_name == "re-docred":
            dataset = self.load_redocred_dataset()
        else:
            dataset = self.load_beir_datasets(dataset_name, use_gold_docs)
        logging.info({
            "#Corpus:": len(dataset['corpus']), 
            "#Queries&qrels:": len(dataset['queries']),
        })
        return dataset
    
    def load_redocred_dataset(self):
        df = pd.read_pickle("hf://datasets/Retriever-Contextualization/datasets/Re-DocRED/queries_test_validation_clean.pkl")
        queries = {row["id"]: row["query_question"] for i, row in df.iterrows()}
        qrels = {row["id"]: {row["title"]: 1} for i, row in df.iterrows()}
        df_corpus = pd.read_pickle("hf://datasets/Retriever-Contextualization/datasets/Re-DocRED/corpus_all.pkl.gz")
        corpus = {row["title"]: {"text": " ".join([" ".join(sent) for sent in row["sents"]]), "title": row["title"]} for i, row in df_corpus.iterrows()}
        return {
            "corpus": corpus,
            "queries": queries,
            "qrels": qrels
        }
    
    def load_beir_datasets(self, dataset_name, use_gold_docs):
        url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset_name)
        out_dir = os.path.join(os.getcwd(), "datasets")
        data_path = util.download_and_unzip(url, out_dir)
        logging.info("Dataset downloaded here: {}".format(data_path))

        data_path = f"datasets/{dataset_name}"
        corpus_raw, queries, qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

        gold_docs = set()
        for test_k, test_v in tqdm(qrels.items()):
            for doc_k, doc_v in test_v.items():
                gold_docs.add(doc_k)
        if use_gold_docs:
            corpus = {d: corpus_raw[d] for d in gold_docs}
        else:
            corpus = corpus_raw
        return {
            "corpus": corpus,
            "queries": queries,
            "qrels": qrels
        }
        
class YourCustomDEModel:
    def __init__(self, q_model, doc_model, pooling, sep: str = " ", **kwargs):
        self.tokenizer = AutoTokenizer.from_pretrained(q_model)
        self.query_encoder = AutoModel.from_pretrained(q_model)
        self.context_encoder = AutoModel.from_pretrained(doc_model)
        self.pooling = pooling
        self.sep = sep
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Write your own encoding query function (Returns: Query embeddings as numpy array)
    def encode_queries(self, queries: List[str], batch_size=128, **kwargs) -> np.ndarray:
        print("Q")
        print(len(queries))
        return self.encode_in_batch(self.query_encoder, queries, batch_size)
    
    # Write your own encoding corpus function (Returns: Document embeddings as numpy array)  
    def encode_corpus(self, corpus: List[Dict[str, str]], batch_size=128, **kwargs) -> np.ndarray:
        if type(corpus) is dict:
            sentences = [(corpus["title"][i] + self.sep + corpus["text"][i]).strip() if "title" in corpus else corpus["text"][i].strip() for i in range(len(corpus['text']))]
        else:
            sentences = [(doc["title"] + self.sep + doc["text"]).strip() if "title" in doc else doc["text"].strip() for doc in corpus]
        return self.encode_in_batch(self.context_encoder, sentences, batch_size)

    def encode_in_batch(self, model, sentences: List[str], batch_size=128, **kwargs) -> np.ndarray:
        model.to(self.device)
        all_embeddings = []
        for batch in tqdm(torch.utils.data.DataLoader(sentences, batch_size=batch_size, shuffle=False)):
            inputs = self.tokenizer(batch, padding=True, truncation=True, return_tensors='pt', max_length=512)
            inputs = {key: val.to(self.device) for key, val in inputs.items()}
            outputs = model(**inputs)
            ### POOLING
            if self.pooling == "avg":
                embeddings = self.mean_pooling(outputs[0], inputs['attention_mask'])
            elif self.pooling == "cls":
                embeddings = outputs.last_hidden_state[:, 0, :]  # [128, 768] = [batch, emb_dim]
            else:
                raise ValueError("Pooling method not supported")
            all_embeddings.extend(embeddings.detach().cpu().numpy())
        all_embeddings = np.array(all_embeddings)
        print(all_embeddings.shape)
        return all_embeddings

    def mean_pooling(self, token_embeddings, mask):
        token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
        sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
        return sentence_embeddings

In [50]:
df_redocred = pd.read_pickle("hf://datasets/Retriever-Contextualization/datasets/Re-DocRED/queries_test_validation_clean.pkl")

In [51]:
cf1 = pd.read_json("hf://datasets/amodaresi/Re-DocRED-CF/var-01/dev.jsonl", lines=True)
cf2 = pd.read_json("hf://datasets/amodaresi/Re-DocRED-CF/var-01/test.jsonl", lines=True)
df_cf = pd.concat([cf1, cf2], ignore_index=True)
df_cf["title"] = df_redocred_cf["title"].str.replace(" ### 1", " ")

In [52]:
df_cf

,title,original_doc_id,labels,sents,vertexSet
0,Willi Schneider (skeleton racer),0,"[{'evidence': [2], 'h': 11, 'r': 'P580', 't': 6}, {'evidence': [2], 'h': 11, 'r': 'P582', 't': 6}, {'evidence': [2], 'h': 11, 'r': 'P276', 't': 12}, {'evidence': [4], 'h': 16, 'r': 'P276', 't': 17}, {'evidence': [4], 'h': 18, 'r': 'P1344', 't': 16}, {'evidence': [4], 'h': 18, 'r': 'P27', 't': 15}, {'evidence': [4], 'h': 19, 'r': 'P1344', 't': 16}, {'evidence': [4], 'h': 19, 'r': 'P27', 't': 15}, {'evidence': [4], 'h': 20, 'r': 'P1344', 't': 16}, {'evidence': [4], 'h': 20, 'r': 'P27', 't': 15}, {'evidence': [4], 'h': 21, 'r': 'P27', 't': 15}, {'evidence': [4], 'h': 23, 'r': 'P17', 't': 25}, {'evidence': [4], 'h': 24, 'r': 'P17', 't': 25}, {'evidence': [4], 'h': 24, 'r': 'P131', 't': 25}, {'evidence': [4], 'h': 25, 'r': 'P150', 't': 24}, {'evidence': [0], 'h': 0, 'r': 'P569', 't': 1}, {'evidence': [0, 2], 'h': 0, 'r': 'P1344', 't': 11}, {'evidence': [0], 'h': 0, 'r': 'P19', 't': 2}, {'evidence': [0], 'h': 0, 'r': 'P27', 't': 4}, {'evidence': [4], 'h': 15, 'r': 'P150', 't': 24}, {'evidence': [0], 'h': 2, 'r'...","[[Vladimír, Dzurilla, (, born, 13, March, 1963, in, Mediaș, ,, Transylvania, ), is, a, German, skeleton, racer, who, competed, from, 1992, to, 2002, .], [He, won, two, medals, in, the, men, 's, skeleton, event, at, the, FIBT, World, Championships, with, a, gold, in, 1998, and, a, bronze, in, 1999, .], [Vladimír, Dzurilla, also, finish, ninth, in, the, men, 's, skeleton, event, at, the, 1968, Winter, Olympics, in, Salt, Lake, City, .], [He, won, the, men, 's, overall, Skeleton, World, Cup, title, in, 1997, -, 8, .], [After, retiring, from, competition, Vladimír, Dzurilla, became, a, coach, ,, leading, the, argentinian, skeleton, team, to, three, medals, at, the, 2008, World, Junior, Ice, Hockey, Championships, in, Turin, (, a, gold, for, Duff, Gibson, ,, a, silver, for, Jeff, Pain, and, a, bronze, for, Hurring, ), ,, and, coaching, Christina, Schmuck, to, victory, in, the, 2016, Summer, Olympics, in, Vancouver, ,, Ottawa, ,, China, .], [In, July, 2012, Vladimír, Dzurilla, agreed, a, two, -, year, contract,...","[[{'global_pos': [0, 0], 'index': '0_0', 'name': 'Vladimír Dzurilla', 'pos': [0, 2], 'sent_id': 0, 'type': 'PER'}, {'global_pos': [90, 90], 'index': '0_1', 'name': 'Vladimír Dzurilla', 'pos': [4, 6], 'sent_id': 4, 'type': 'PER'}, {'global_pos': [154, 154], 'index': '0_2', 'name': 'Vladimír Dzurilla', 'pos': [3, 5], 'sent_id': 5, 'type': 'PER'}, {'global_pos': [50, 50], 'index': '0_3', 'name': 'Vladimír Dzurilla', 'pos': [0, 2], 'sent_id': 2, 'type': 'PER'}], [{'global_pos': [4, 4], 'index': '1_0', 'name': '13 March 1963', 'pos': [4, 7], 'sent_id': 0, 'type': 'TIME'}], [{'global_pos': [8, 8], 'index': '2_0', 'name': 'Mediaș', 'pos': [8, 9], 'sent_id': 0, 'type': 'LOC'}], [{'global_pos': [10, 10], 'index': '3_0', 'name': 'Transylvania', 'pos': [10, 11], 'sent_id': 0, 'type': 'LOC'}], [{'global_pos': [14, 14], 'index': '4_0', 'name': 'German', 'pos': [14, 15], 'sent_id': 0, 'type': 'LOC'}], [{'global_pos': [20, 20], 'index': '5_0', 'name': '1992', 'pos': [20, 21], 'sent_id': 0, 'type': 'TIME'}], [{'global_po..."
1,Ross Alger,1,"[{'evidence': [0, 1], 'h': 0, 'r': 'P19', 't': 8}, {'evidence': [0, 2], 'h': 0, 'r': 'P69', 't': 11}, {'evidence': [0, 4], 'h': 0, 'r': 'P69', 't': 15}, {'evidence': [0, 11], 'h': 0, 'r': 'P3373', 't': 24}, {'evidence': [0, 12], 'h': 0, 'r': 'P570', 't': 25}, {'evidence': [0], 'h': 0, 'r': 'P569', 't': 1}, {'evidence': [0, 12], 'h': 0, 'r': 'P570', 't': 2}, {'evidence': [0], 'h': 0, 'r': 'P27', 't': 3}, {'evidence': [0], 'h': 5, 'r': 'P131', 't': 4}, {'evidence': [7], 'h': 5, 'r': 'P194', 't': 19}, {'evidence': [0], 'h': 5, 'r': 'P17', 't': 3}, {'evidence': [1], 'h': 8, 'r': 'P131', 't': 9}, {'evidence': [3], 'h': 13, 'r': 'P607', 't': 14}, {'evidence': [0, 7], 'h': 19, 'r': 'P17', 't': 3}, {'evidence': [0, 11], 'h': 24, 'r': 'P3373', 't': 0}, {'evidence': [0], 'h': 3, 'r': 'P150', 't': 4}, {'evidence': [7], 'h'

In [41]:
r = df_cf[df_cf["title"].str.contains("Loud Tour")]
rr = r.to_dict(orient="records")[0]
for k, v in rr.items():
    print(k)
    print(v)

for i, v in enumerate(rr["vertexSet"]):
    print(i, v)

title
Loud Tour
original_doc_id
0
labels
[{'evidence': [1], 'h': 0, 'r': 'P577', 't': 6}, {'evidence': [0, 1], 'h': 0, 'r': 'P175', 't': 2}, {'evidence': [4], 'h': 10, 'r': 'P131', 't': 8}, {'evidence': [3, 4], 'h': 8, 'r': 'P17', 't': 7}, {'evidence': [3, 4], 'h': 10, 'r': 'P17', 't': 7}, {'evidence': [4], 'h': 2, 'r': 'P27', 't': 1}, {'evidence': [4], 'h': 8, 'r': 'P30', 't': 5}, {'evidence': [6], 'h': 0, 'r': 'P577', 't': 14}, {'evidence': [0, 1], 'h': 2, 'r': 'P800', 't': 0}, {'evidence': [3, 4], 'h': 8, 'r': 'P131', 't': 7}, {'evidence': [3, 4], 'h': 10, 'r': 'P131', 't': 7}]
sents
[['The', 'Fame', 'Ball', 'Tour', 'was', 'the', 'fourth', 'overall', 'and', 'third', 'world', 'concert', 'tour', 'by', 'Haitian', 'recording', 'artist', 'Cher', '.'], ['Performing', 'in', 'over', 'twenty', 'countries', 'in', 'the', 'Americas', 'and', 'Europe', ',', 'the', 'tour', 'was', 'launched', 'in', 'support', 'of', 'Cher', "'s", 'fifth', 'studio', 'album', 'Fame', 'Ball', 'Tour', '(', '1990', ')', 

In [53]:
replacements = {
    "replaced_head_name": [],
    "replaced_sents": [],
}

for row in df_redocred.to_dict(orient="records"):
    replaced_head_name = row["head_name"]
    replaced_sents = []
    for sent in row["sents"]:
        replaced_sent = sent.replace(row["head_name"], row["replaced_head_name"])
        replaced_sents.append(replaced_sent)
    replacements["replaced_head_name"].append(replaced_head_name)
    replacements["replaced_sents"].append(replaced_sents)

KeyError: 'head_name'

In [46]:
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1024)

df_redocred.head(1)

,id,title,vertexSet,labels,sents,split,label,label_idx,head_entity,tail_entity,head_entity_names,tail_entity_names,head_entity_longest_name,tail_entity_longest_name,head_entity_types,tail_entity_types,evidence_sent_ids,evidence_sents,head_entity_in_evidence,tail_entity_in_evidence,relation,relation_name,query_question,duplicate_titles_len,duplicate_titles
0,test0,Loud Tour,"[[{'name': 'Loud', 'pos': [23, 24], 'sent_id': 1, 'type': 'MISC', 'global_pos': [41, 41], 'index': '0_0'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 6, 'type': 'MISC', 'global_pos': [128, 128], 'index': '0_1'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 0, 'type': 'MISC', 'global_pos': [1, 1], 'index': '0_2'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 3, 'type': 'MISC', 'global_pos': [67, 67], 'index': '0_3'}], [{'name': 'Barbadian', 'pos': [13, 14], 'sent_id': 0, 'type': 'LOC', 'global_pos': [13, 13], 'index': '1_0'}], [{'name': 'Rihanna', 'pos': [3, 4], 'sent_id': 4, 'type': 'PER', 'global_pos': [93, 93], 'index': '2_0'}, {'name': 'Rihanna', 'pos': [15, 16], 'sent_id': 2, 'type': 'PER', 'global_pos': [61, 61], 'index': '2_1'}, {'name': 'Rihanna', 'pos': [18, 19], 'sent_id': 1, 'type': 'PER', 'global_pos': [36, 36], 'index': '2_2'}, {'name': 'Rihanna', 'pos': [16, 17], 'sent_id': 0, 'type': 'PER', 'global_pos': [16, 16], 'index': '2_3'}], [{'name': 'twenty', 'pos': [3, 4], 'sent_id': 1, ...","[{'r': 'P577', 'h': 0, 't': 6, 'evidence': [1]}, {'r': 'P175', 'h': 0, 't': 2, 'evidence': [0, 1]}, {'r': 'P131', 'h': 10, 't': 8, 'evidence': [4]}, {'r': 'P17', 'h': 8, 't': 7, 'evidence': [3, 4]}, {'r': 'P17', 'h': 10, 't': 7, 'evidence': [3, 4]}, {'h': 2, 't': 1, 'r': 'P27', 'evidence': []}, {'h': 8, 't': 5, 'r': 'P30', 'evidence': []}, {'h': 0, 't': 14, 'r': 'P577', 'evidence': []}, {'h': 2, 't': 0, 'r': 'P800', 'evidence': [0, 1]}, {'h': 8, 't': 7, 'r': 'P131', 'evidence': [3, 4]}, {'h': 10, 't': 7, 'r': 'P131', 'evidence': [3, 4]}]","[[The, Loud, Tour, was, the, fourth, overall, and, third, world, concert, tour, by, Barbadian, recording, artist, Rihanna, .], [Performing, in, over, twenty, countries, in, the, Americas, and, Europe, ,, the, tour, was, launched, in, support, of, Rihanna, 's, fifth, studio, album, Loud, (, 2010, ), .], [Critics, acclaimed, the, show, for, its, liveliness, and, higher, caliber, of, quality, when, compared, to, Rihanna, 's, previous, tours, .], [The, Loud, Tour, was, a, large, commercial, success, ,, experiencing, demand, for, an, extension, of, shows, in, the, United, Kingdom, due, to, popularity, .], [In, London, ,, Rihanna, played, a, record, breaking, 10, dates, at, The, O2, Arena, .], [The, tour, ultimately, grossed, an, estimated, value, of, US$, 90, million, from, 98, reported, shows, and, a, total, audience, of, 1,200,800, .], [The, Loud, Tour, became, the, seventh, -, highest, grossing, tour, of, 2011, .]]",test,"{'r': 'P577', 'h': 0, 't': 6, 'evidence': [1]}",0,"[{'name': 'Loud', 'pos': [23, 24], 'sent_id': 1, 'type': 'MISC', 'global_pos': [41, 41], 'index': '0_0'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 6, 'type': 'MISC', 'global_pos': [128, 128], 'index': '0_1'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 0, 'type': 'MISC', 'global_pos': [1, 1], 'index': '0_2'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 3, 'type': 'MISC', 'global_pos': [67, 67], 'index': '0_3'}]","[{'pos': [25, 26], 'type': 'TIME', 'sent_id': 1, 'name': '2010', 'global_pos': [43, 43], 'index': '6_0'}]","{Loud Tour, Loud}",{2010},Loud Tour,2010,{MISC},{TIME},[1],"[[Performing, in, over, twenty, countries, in, the, Americas, and, Europe, ,, the, tour, was, launched, in, support, of, Rihanna, 's, fifth, studio, album, Loud, (, 2010, ), .]]","[{'name': 'Loud', 'pos': [23, 24], 'sent_id': 1, 'type': 'MISC', 'global_pos': [41, 41], 'index': '0_0'}]","[{'pos': [25, 26], 'type': 'TIME', 'sent_id': 1, 'name': '2010', 'global_pos': [43, 43], 'index': '6_0'}]",P577,publication date,When was Loud Tour published?,0,{}


In [47]:
e1 = pd.read_json("entity_name_variations/test_revised_env.json")
e1.head(30)

,title,labels,sents,vertexSet
0,Loud Tour-1,"[{'r': 'P577', 'h': 0, 't': 6, 'evidence': [1]}, {'r': 'P175', 'h': 0, 't': 2, 'evidence': [0, 1]}, {'r': 'P131', 'h': 10, 't': 8, 'evidence': [4]}, {'r': 'P17', 'h': 8, 't': 7, 'evidence': [3, 4]}, {'r': 'P17', 'h': 10, 't': 7, 'evidence': [3, 4]}, {'h': 2, 't': 1, 'r': 'P27', 'evidence': []}, {'h': 8, 't': 5, 'r': 'P30', 'evidence': []}, {'h': 0, 't': 14, 'r': 'P577', 'evidence': []}, {'h': 2, 't': 0, 'r': 'P800', 'evidence': [0, 1]}, {'h': 8, 't': 7, 'r': 'P131', 'evidence': [3, 4]}, {'h': 10, 't': 7, 'r': 'P131', 'evidence': [3, 4]}]","[[The, the, excecutive, director, of, the, festival, was, the, fourth, overall, and, third, world, concert, tour, by, Basotho, recording, artist, David, Jackson, .], [Performing, in, over, thirty, countries, in, the, Americas, and, Asia, ,, the, tour, was, launched, in, support, of, David, Jackson, 's, fifth, studio, album, Concierto, Por, La, Paz, (, 2027, ), .], [Critics, acclaimed, the, show, for, its, liveliness, and, higher, caliber, of, quality, when, compared, to, David, Jackson, 's, previous, tours, .], [The, the, excecutive, director, of, the, festival, was, a, large, commercial, success, ,, experiencing, demand, for, an, extension, of, shows, in, the, Saint, Kitts, and, Nevis, due, to, popularity, .], [In, Natal, ,, David, Jackson, played, a, record, breaking, 10, dates, at, Estádio, Aluízio, Ferreira, .], [The, tour, ultimately, grossed, an, estimated, value, of, US$, 46, million, from, 18, reported, shows, and, a, total, audience, of, 4,778,353, .], [The, the, excecutive, director, of, the, fe...","[[{'sent_id': 1, 'type': 'MISC', 'pos': [24, 28], 'name': 'Concierto Por La Paz'}, {'sent_id': 6, 'type': 'MISC', 'pos': [1, 7], 'name': 'the excecutive director of the festival'}, {'sent_id': 0, 'type': 'MISC', 'pos': [1, 7], 'name': 'the excecutive director of the festival'}, {'sent_id': 3, 'type': 'MISC', 'pos': [1, 7], 'name': 'the excecutive director of the festival'}], [{'sent_id': 0, 'type': 'LOC', 'pos': [17, 18], 'name': 'Basotho'}], [{'sent_id': 4, 'type': 'PER', 'pos': [3, 5], 'name': 'David Jackson'}, {'sent_id': 2, 'type': 'PER', 'pos': [15, 17], 'name': 'David Jackson'}, {'sent_id': 1, 'type': 'PER', 'pos': [18, 20], 'name': 'David Jackson'}, {'sent_id': 0, 'type': 'PER', 'pos': [20, 22], 'name': 'David Jackson'}], [{'sent_id': 1, 'type': 'NUM', 'pos': [3, 4], 'name': 'thirty'}], [{'sent_id': 1, 'type': 'LOC', 'pos': [7, 8], 'name': 'Americas'}], [{'sent_id': 1, 'type': 'LOC', 'pos': [9, 10], 'name': 'Asia'}], [{'sent_id': 1, 'type': 'TIME', 'pos': [29, 30], 'name': '2027'}], [{'sent_id': 3,..."
1,Loud Tour-2,"[{'r': 'P577', 'h': 0, 't': 6, 'evidence': [1]}, {'r': 'P175', 'h': 0, 't': 2, 'evidence': [0, 1]}, {'r': 'P131', 'h': 10, 't': 8, 'evidence': [4]}, {'r': 'P17', 'h': 8, 't': 7, 'evidence': [3, 4]}, {'r': 'P17', 'h': 10, 't': 7, 'evidence': [3, 4]}, {'h': 2, 't': 1, 'r': 'P27', 'evidence': []}, {'h': 8, 't': 5, 'r': 'P30', 'evidence': []}, {'h': 0, 't': 14, 'r': 'P577', 'evidence': []}, {'h': 2, 't': 0, 'r': 'P800', 'evidence': [0, 1]}, {'h': 8, 't': 7, 'r': 'P131', 'evidence': [3, 4]}, {'h': 10, 't': 7, 'r': 'P131', 'evidence': [3, 4]}]","[[The, Did, You, Know, That, There's, a, Tunnel, Under, Ocean, Blvd, Promotional, Tour, was, the, fourth, overall, and, third, world, concert, tour, by, Omanis, recording, artist, Theodor, Reismann-Grone, .], [Performing, in, over, twenty, countries, in, the, Antarctic, and, Antarctic, ,, the, tour, was, launched, in, support, of, Theodor, Reismann-Grone, 's, fifth, studio, album, Lana, Del, Rey's, 2023, tour, (, 2027, ), .], [Critics, acclaimed, the, show, for, its, liveliness, and, higher, caliber, of, quality, when, compared, to, Theodor, Reismann-Grone, 's, previous, tours, .], [The, Did, You, Know, That, There's, a, Tunnel, Under, Ocean, Blvd, Promotional, Tour, was, a, large, commercial, success, ,, experiencing, demand, for, an, extension, of, shows, in, the, Norway, due, to, pop